In [1]:
import os
import pandas as pd

# Descrevendo as variáveis de cada tabela

### <ins>Furto de Celulares, Roubo de Celulares, Furto de Veículos e roubo de veículos</ins>


| COLUNA                   | TIPO | DESCRICAO |
|--------------------------|------|-----------|
| ANO_BO                   | teste     |           |
| NUM_BO                   |      |           |
| NUMERO_BOLETIM           |      |           |
| BO_INICIADO              |      |           |
| BO_EMITIDO               |      |           |
| DATAOCORRENCIA           |      |           |
| HORAOCORRENCIA           |      |           |
| PERIDOOCORRENCIA         |      |           |
| DATACOMUNICACAO          |      |           |
| DATAELABORACAO           |      |           |
| BO_AUTORIA               |      |           |
| FLAGRANTE                |      |           |
| NUMERO_BOLETIM_PRINCIPAL |      |           |
| LOGRADOURO               |      |           |
| NUMERO                   |      |           |
| BAIRRO                   |      |           |
| CIDADE                   |      |           |
| UF                       |      |           |
| LATITUDE                 |      |           |
| LONGITUDE                |      |           |
| DESCRICAOLOCAL           |      |           |
| EXAME                    |      |           |
| SOLUCAO                  |      |           |
| DELEGACIA_NOME           |      |           |
| DELEGACIA_CIRCUNSCRICAO  |      |           |
| ESPECIE                  |      |           |
| RUBRICA                  |      |           |
| DESDOBRAMENTO            |      |           |
| STATUS                   |      |           |
| TIPOPESSOA               |      |           |
| VITIMAFATAL              |      |           |
| NATURALIDADE             |      |           |
| NACIONALIDADE            |      |           |
| SEXO                     |      |           |
| DATANASCIMENTO           |      |           |
| IDADE                    |      |           |
| ESTADOCIVIL              |      |           |
| PROFISSAO                |      |           |
| GRAUINSTRUCAO            |      |           |
| CORCUTIS                 |      |           |
| NATUREZAVINCULADA        |      |           |
| TIPOVINCULO              |      |           |
| RELACIONAMENTO           |      |           |
| PARENTESCO               |      |           |
| PLACA_VEICULO            |      |           |
| UF_VEICULO               |      |           |
| CIDADE_VEICULO           |      |           |
| DESCR_COR_VEICULO        |      |           |
| DESCR_MARCA_VEICULO      |      |           |
| ANO_FABRICACAO           |      |           |
| ANO_MODELO               |      |           |
| DESCR_TIPO_VEICULO       |      |           |
| QUANT_CELULAR            |      |           |
| MARCA_CELULAR            |      |           |

### Feminicidio, Homicídio, Latrocínio, LCSM, MDIP e Morte Suspeita

<img src="equiColunas.png" alt="Alternative text" />